# Analysis 분석 순서
## 1. Train_data (9973 경기 = 99730 row) *Encoding = utf8
1) 승패에 영향을 주는 지표 (Kill, Death, Assist, Damage 등) 탐색하기

2) EDA를 거치면서 분석에 영향을 줄만한 지표를 만들기 (Ward, KDA2)

3) 앞에서 구한 지표에 머신러닝을 이용하여 Weight와 Bias 학습하기 {model : argmax[ X(1x5) \* W(5x2) + b(1x2) ]}

4) 기타
- 최종 발표 당시) 발표시점에서는 약 6800경기를 이용하여 Kill, Death, Assist, Damage, Ward, KDA2 지표를 학습함.
- 주의) Encoding=utf8으로 되어있어서 엑셀을 사용할 경우 파일이 깨져서 보일 수 있음. (메모장 이용을 추천)
- 주의) Encoding=utf8이 아닌 파일을 read_csv로 열 경우 오류가 생김

### 2. Test_data (1308 경기 = 13080 row) *Encoding = 기본값
1) 머신러닝 결과를 이용해서 테스트를 진행

2) 기타

- 3) Predict와 차이점은 Test는 이미 결과가 나온 경기를 이용하나 3) Predict는 어떤 경기를 기준으로 게임이 시작하기 전이라고 보고 평균 지표를 계산하여 예측한 후 그 경기 결과와 비교하는 것임.
- 주의) Encoding=기본값으로 되어있어서 read_csv에서 utf8로 Encoding된 것도 열 수 있으나 깨진 상태로 열림

### 3. Predict (Our Goal) - (아직 일어나지 않는 경기를 만들어서 예측)
1) 초기 목표는 프로게이머들의 개인전적, 대회전적 등을 분석해서 각 선수가 게임의 승패에 미치는 영향을 알아내어 이적시장에서 최적의 팀을 구성하는 것이 목표였으나, 현재는 여러가지 문제로 진행이 되지 않고, 최상위권 아마추어 유저들과 프로유저들의 평균 지표를 계산하여 이를 앞에서 머신러닝을 통해 학습한 Weight와 Bias를 가지고 예측해보는 것을 진행함.

2) 설명
- 최종 발표 당시) 26개의 무작위로 선정한 경기를 기준으로 정해진 값이 아닌 모든 지표를 게임에 참여한 선수들의 최근 20경기 내의 전적의 평균 지표를 이용하여 계산하였고, 이를 가지고 평가함.
- 최종 발표 당시) Test1(K,D,A,W) : 정답 14, 오답 7, 무승부 5
- 최종 발표 당시) Test2(W,Da,KDA2) : 정답 14, 오답 6, 무승부 6
- 정답은 결과를 정확하게 예측한 경우, 오답은 결과를 정확하게 틀린 경우, 데이터를 양쪽 같은 포지션 선수들의 지표의 차이를 가지고 학습을 하는데, 거기에 -를 모두 곱한 경우(경기가 승패를 기준으로 대칭구조이므로 뒤집은 경우) 결과를 넣어서 한 경기를 2번 예측시켜서 나온 결과를 사용
- EX) (승) 선수1, 선수2, 선수3, 선수4, 선수5 - (패) 선수6, 선수7, 선수8, 선수9, 선수10
- EX) (패) 선수6, 선수7, 선수8, 선수9, 선수10 - (승) 선수1, 선수2, 선수3, 선수4, 선수5
- EX) 학습에 사용된 지표 차: 선수1 and 선수6 (same position in game), (지표1 = 선수1.지표1 - 선수6.지표1) and (지표1' = 선수6.지표1 - 선수1.지표1) 이렇게 1경기마다 뒤집어서 2번 넣음

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

## 1. Train data (9973 경기 = 99730 row)
### 0) Data Load (Train_data.zip 3,043KB)

In [169]:
# get data file names
path =r'C:\Users\waf\Documents\00_DGIST 수업\03_02\Big data\프로젝트\2017_BigData\Data\all_data_utf'
filenames = glob.glob(path + "/*.csv")
filenames
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding="utf8", engine='python'))
# Concatenate all data into one DataFrame
kda_data = pd.concat(dfs, ignore_index=True)
kda_data['Death_fake'] = kda_data['Death']
kda_data['Death_fake'] = kda_data['Death_fake'].replace(0, 1)
kda_data['KDA'] = (kda_data['Kill'] + kda_data['Assist'])/ kda_data['Death_fake']
kda_data = kda_data.drop('Death_fake', 'columns')
kda_data

,Unnamed: 0,MatchNumber,GameResult,Champion,Nickname,Kill,Death,Assist,Damage,Buy_Pward,Build_ward,Destroy_ward,CS,CSperMin,Baron,Dragon,Tower,KDA
0,0,30220,패배,마오카이,twice손채영,7,9,6,20924,2,24,2,149,5.5,0,1,2,1.44444
1,1,30220,패배,세주아니,작전동 노란머리,6,7,11,12273,6,28,6,130,4.8,0,1,2,2.42857
2,2,30220,패배,야스오,hf ghkyu,10,7,7,23123,3,12,2,210,7.7,0,1,2,2.42857
3,3,30220,패배,진,BtC RYAN,3,11,6,9015,1,10,6,164,6.0,0,1,2,0.818182
4,4,30220,패배,미스 포츈,커소리어스나,3,9,12,16067,7,35,3,42,1.5,0,1,2,1.66667
5,5,30220,승리,카밀,Heiz4,5,9,9,20328,0,11,9,196,7.2,1,1,10,1.55556
6,6,30220,승리,니달리,담원 크러쉬,11,4,13,23256,5,16,9,141,5.2,1,1,10,6
7,7,30220,승리,트위스티드 페이트,TeachersWu,13,3,8,24899,4,13,5,255,9.4,1,1,10,7
8,8,30220,승리,이즈리얼,해강고원딜 팬,10,7,15,32430,1,17,2,146,5.4,1,1,10,3.57143
9,9,30220,승리,카르마,Heart,4,6,19,15573,5,17,7,58,2.1,1,1,10,3.83333


### 1) 승패에 영향을 줄만한 지표 탐색
- 각 팀별로 특정 지표의 단순 합을 계산하여 높은 쪽이 이긴다고 예측하여 지표를 탐색
- more_win : 지표의 합이 높은 쪽이 이긴 경우
- more_lose : 지표의 합이 높은 쪽이 진 경우
- not_more : 지표의 합이 같은 경우

In [24]:
def Sum_predict(parameter):
    more_win = 0
    more_lose = 0
    not_more = 0
    total = 0
    tmp = []
    index = 0
    for i in kda_data.groupby(['MatchNumber','GameResult'])[parameter].sum():
        tmp.append(i)
        index += 1
        if index == 2:
            total += 1
            if tmp[0] > tmp[1]:
                more_win += 1
            elif tmp[0] < tmp[1]:
                more_lose += 1
            else:
                not_more += 1
            tmp = []
            index = 0
    # print(more_kill_win/total)
    print("more win  :",more_win/total)
    print("more lose :",more_lose/total)
    print("not more  :",not_more/total)

### 1-1) Kill Sum

In [25]:
Sum_predict("Kill")

more win  : 0.9465557003910559
more lose : 0.039506668003609746
not more  : 0.013937631605334403


### 1-2) Assist Sum

In [27]:
Sum_predict("Assist")

more win  : 0.9294094053945653
more lose : 0.06357164343728065
not more  : 0.007018951168154016


### 1-3) KDA Sum [(K+A)/D]

In [28]:
Sum_predict("KDA")

more win  : 0.9495638223202647
more lose : 0.05043617767973529
not more  : 0.0


### 1-4) Damage Sum

In [30]:
Sum_predict("Damage")

more win  : 0.8752632106688057
more lose : 0.12473678933119423
not more  : 0.0


### 1-5) Buy_Pward Sum

In [31]:
Sum_predict("Buy_Pward")

more win  : 0.5449714228416725
more lose : 0.3916574751829941
not more  : 0.0633711019753334


### 1-6) Build_ward

In [33]:
Sum_predict("Build_ward")

more win  : 0.571041812894816
more lose : 0.4058959189812494
not more  : 0.023062268123934623


### 1-7) Destroy_ward

In [34]:
Sum_predict("Destroy_ward")

more win  : 0.5921989371302517
more lose : 0.3620776095457736
not more  : 0.04572345332397473


### 1-8) CS

In [35]:
Sum_predict("CS")

more win  : 0.814699689160734
more lose : 0.17978542063571643
not more  : 0.005514890203549584


### 1-9) Baron

In [36]:
Sum_predict("Baron")

more win  : 0.5298305424646546
more lose : 0.06838463852401484
not more  : 0.4017848190113306


### 1-10) Dragon

In [37]:
Sum_predict("Dragon")

more win  : 0.6211771783816304
more lose : 0.158828837862228
not more  : 0.21999398375614157


### 1-11) Tower

In [38]:
Sum_predict("Tower")

more win  : 0.9666098465857816
more lose : 0.01574250476285972
not more  : 0.017647648651358667


### 2) 분석에 도움이 될만한 새로운 지표 만들기

### 2-1) KDA2 Sum [(K+0.2A-1.2D] (몇 번의 시도 끝에 가장 높다고 생각하는 비율을 만듬)

In [171]:
kda_data['KDA2'] = kda_data['Kill'] + 0.2*kda_data['Assist'] - 1.2*kda_data['Death']
Sum_predict("KDA2")

more win  : 0.9582873759149704
more lose : 0.04151208262308232
not more  : 0.0002005414619472576


### 2-2) Ward (0.18\*Buy_Pward+Build_ward+2\*Destroy_ward) [몇번의 시도 끝에 높은 수치]

In [172]:
kda_data["Ward"] = 0.18*kda_data["Buy_Pward"] + kda_data["Build_ward"] + 2*kda_data["Destroy_ward"]
Sum_predict("Ward")

more win  : 0.7212473678933119
more lose : 0.2782512784518199
not more  : 0.000501353654868144


### 2-3) Object (3.2\*Baron+Dragon) [몇번의 시도 끝에 높은 수치]

In [173]:
kda_data["Object"] = 3.2*kda_data["Baron"] + 1*kda_data["Dragon"]
Sum_predict("Object")

more win  : 0.763361074902236
more lose : 0.11932216985861827
not more  : 0.1173167552391457


### 3) 앞에서 탐색한 지표로 머신러닝으로 W, b 구하기
### 3-1) data preprocess

In [174]:
sub = kda_data[["MatchNumber","GameResult","Kill","Death","Assist","Ward","Damage","KDA2","Tower","Object","CS"]]
result = []

for i in sub["GameResult"]:
    if i == "승리":
        result.append(1)
    else:
        result.append(0)
        
sub["Result"] = result;
sub.head(20)

C:\Users\waf\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,MatchNumber,GameResult,Kill,Death,Assist,Ward,Damage,KDA2,Tower,Object,CS,Result
0,30220,패배,7,9,6,28.36,20924,-2.6,2,1,149,0
1,30220,패배,6,7,11,41.08,12273,-0.2,2,1,130,0
2,30220,패배,10,7,7,16.54,23123,3,2,1,210,0
3,30220,패배,3,11,6,22.18,9015,-9,2,1,164,0
4,30220,패배,3,9,12,42.26,16067,-5.4,2,1,42,0
5,30220,승리,5,9,9,29,20328,-4,10,4.2,196,1
6,30220,승리,11,4,13,34.9,23256,8.8,10,4.2,141,1
7,30220,승리,13,3,8,23.72,24899,11,10,4.2,255,1
8,30220,승리,10,7,15,21.18,32430,4.6,10,4.2,146,1
9,30220,승리,4,6,19,31.9,15573,0.6,10,4.2,58,1


In [134]:
def sigmoid(n):
    return 1 / (1 + np.exp(-n))

In [82]:
def cvt(parameter):
    index = 0
    result = []
    tmp = []
    for item in sub[parameter]:
        index += 1
        tmp.append(item)
        if index % 10 == 0:
            result.append(tmp)
            tmp=[]
    result = np.array(result)
    res = np.ones([result.shape[0],int(result.shape[1]/2)])
    for i in range(len(result)):
        res[i] = result[i][0:5]-result[i][5:10]
    return res

In [106]:
def cvt_sum(parameter): # 10명의 전체 합에 대한 비율 * 100
    index = 0
    result = []
    tmp = []
    for item in sub[parameter]:
        index += 1
        tmp.append(item)
        if index % 10 == 0:
            sum_da = 0
            for j in tmp:
                sum_da += j
            result.append(np.array(tmp)/sum_da*100)
            tmp=[]
    result = np.array(result)
    res = np.ones([result.shape[0],int(result.shape[1]/2)])
    for i in range(len(result)):
        res[i] = result[i][0:5]-result[i][5:10]
    return res

In [ ]:
def cvt_result(parameter): # 결과를 뽑을 때 사용
    index = 0
    result = []
    tmp = []
    for item in sub[parameter]:
        index += 1
        if index % 10 == 1:
            tmp.append(item)
        if index % 10 == 6:
            tmp.append(item)
        if index % 10 == 0:
            result.append(tmp)
            tmp=[]
    result = np.array(result)
    return result

In [175]:
kill = cvt("Kill")
death = cvt("Death")
assist = cvt("Assist")
kda2 = cvt("KDA2")
ward = cvt_sum("Ward")
damage = cvt_sum("Damage")
tower = cvt("Tower")
obj = cvt("Object")
CS = cvt_sum("CS")
ylabel = cvt_result("Result")
actual = np.argmax(ylabel,axis=1)
actual.shape

(9973,)

### 3-2) Machine Learning
- X는 10명의 데이터를 5명, 5명 팀으로 나눈 후 같은 포지션의 지표를 뺀 값
- argmax(X(1x5) \* W(5x2) + b(1x2))

In [126]:
trait = 3000 # 학습 횟수
train_list = [kill,death,assist,kda2,ward,damage,tower,obj,CS]
for i,data in enumerate(train_list):
    x_data = data
    y_data = ylabel
    
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)

    W = tf.Variable(tf.random_uniform([5, 2], -1., 1.))
    b = tf.Variable(tf.zeros([2]))

    L = tf.add(tf.matmul(X, W), b)
    L = tf.nn.relu(L)
    model = tf.nn.softmax(L)

    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    for step in range(trait):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        
        if (step + 1) == 1:
            print(step + 1, "cost:", sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        if (step + 1) % (trait/10) == 0:
            print(step + 1, "cost:", sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    prediction = tf.argmax(model, 1)
    target = tf.argmax(Y, 1)
    print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
    print('실제값:', sess.run(target, feed_dict={Y: y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))
    
    if i == 0:
        w1 = sess.run(W)
        b1 = sess.run(b)
#         a1 = sess.run(accuracy)
    elif i == 1:
        w2 = sess.run(W)
        b2 = sess.run(b)
#         a2 = sess.run(accuracy)
    elif i == 2:
        w3 = sess.run(W)
        b3 = sess.run(b)
#         a3 = sess.run(accuracy)
    elif i == 3:
        w4 = sess.run(W)
        b4 = sess.run(b)
#         a4 = sess.run(accuracy)
    elif i == 4:
        w5 = sess.run(W)
        b5 = sess.run(b)
#         a5 = sess.run(accuracy)
    elif i == 5:
        w6 = sess.run(W)
        b6 = sess.run(b)
#         a6 = sess.run(accuracy)
    elif i == 6:
        w7 = sess.run(W)
        b7 = sess.run(b)
#         a7 = sess.run(accuracy)
    elif i == 7:
        w8 = sess.run(W)
        b8 = sess.run(b)
#         a8 = sess.run(accuracy)
    elif i == 8:
        w9 = sess.run(W)
        b9 = sess.run(b)
#         a9 = sess.run(accuracy)

1 cost: 1.46767
300 cost: 0.172476
600 cost: 0.134929
900 cost: 0.133602
1200 cost: 0.133003
1500 cost: 0.132717
1800 cost: 0.132485
2100 cost: 0.132443
2400 cost: 0.132443
2700 cost: 0.132443
3000 cost: 0.132443
예측값: [1 0 0 ..., 1 1 0]
실제값: [1 0 0 ..., 1 1 0]
정확도: 95.44
1 cost: 0.45803
300 cost: 0.14046
600 cost: 0.139554
900 cost: 0.139521
1200 cost: 0.139521
1500 cost: 0.139521
1800 cost: 0.139521
2100 cost: 0.139521
2400 cost: 0.139521
2700 cost: 0.139521
3000 cost: 0.139521
예측값: [1 0 0 ..., 1 1 0]
실제값: [1 0 0 ..., 1 1 0]
정확도: 95.14
1 cost: 0.878693
300 cost: 0.194885
600 cost: 0.191024
900 cost: 0.190831
1200 cost: 0.190831
1500 cost: 0.190831
1800 cost: 0.190831
2100 cost: 0.190831
2400 cost: 0.190831
2700 cost: 0.190831
3000 cost: 0.190831
예측값: [1 0 0 ..., 1 1 0]
실제값: [1 0 0 ..., 1 1 0]
정확도: 93.31
1 cost: 1.7505
300 cost: 0.115569
600 cost: 0.114591
900 cost: 0.114312
1200 cost: 0.114261
1500 cost: 0.114251
1800 cost: 0.114251
2100 cost: 0.114251
2400 cost: 0.114251
2700 cost: 0

In [177]:
predict = []
for i in range(int(len(sub)/10)):
    predict.append(np.argmax(0.9544*sigmoid(np.matmul(kill[i],w1)+b1)+0.9514*sigmoid(np.matmul(death[i],w2)+b2)
                             +0.9331*sigmoid(np.matmul(assist[i],w3)+b3)+0.9608*sigmoid(np.matmul(kda2[i],w4)+b4)
                             +0.7308*sigmoid(np.matmul(ward[i],w5)+b5)+0.8963*sigmoid(np.matmul(damage[i],w6)+b6)
                             +0.9740*sigmoid(np.matmul(tower[i],w7)+b7)+0.8267*sigmoid(np.matmul(obj[i],w8)+b8)
                             +0.8258*sigmoid(np.matmul(CS[i],w9)+b9)))

score = 0
for i in range(len(predict)):
    if predict[i] == actual[i]:
        score += 1
score = score/len(predict)
print("Train_data 정확도 :",score)

Train_data 정확도 : 0.9719241953273839


## 2. Test_data (1308 경기 = 13080 row)
- 머신러닝 결과 확인용
- Train_data와 Encoding이 다름을 주의

In [162]:
# get data file names
path =r'C:\Users\waf\Documents\00_DGIST 수업\03_02\Big data\프로젝트\2017_BigData\Data\all_data'
filenames = glob.glob(path + "/*.csv")
filenames
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, engine='python'))
# Concatenate all data into one DataFrame
test_data = pd.concat(dfs, ignore_index=True)
test_data['Death_fake'] = test_data['Death']
test_data['Death_fake'] = test_data['Death_fake'].replace(0, 1)
test_data['KDA'] = (test_data['Kill'] + test_data['Assist'])/ test_data['Death_fake']
test_data = test_data.drop('Death_fake', 'columns')
test_data

,Unnamed: 0,MatchNumber,GameResult,Champion,Nickname,Kill,Death,Assist,Damage,Buy_Pward,Build_ward,Destroy_ward,CS,CSperMin,Baron,Dragon,Tower,KDA
0,0,154169,패배,오른,Fly,0,6,3,10743,1,7,1,112,5.1,0,0,1,0.500000
1,1,154169,패배,리 신,1112213311,2,3,5,6993,8,26,6,101,4.6,0,0,1,2.333333
2,2,154169,패배,아지르,웨이 지안,3,4,1,10188,3,12,4,187,8.5,0,0,1,1.000000
3,3,154169,패배,미스 포츈,ARE2A,5,8,4,15458,1,6,6,150,6.8,0,0,1,1.125000
4,4,154169,패배,쉔,혼자가 편해졌어,2,4,9,5339,5,17,2,45,2.0,0,0,1,2.750000
5,5,154169,승리,갱플랭크,bbq Crazy,5,0,10,21246,2,14,7,221,10.0,0,0,5,15.000000
6,6,154169,승리,이블린,jinwandalaohu,12,2,4,12514,1,18,1,91,4.1,0,0,5,8.000000
7,7,154169,승리,조이,whisper snow,0,1,9,10661,3,22,3,156,7.1,0,0,5,9.000000
8,8,154169,승리,바루스,Labixiaoxin,7,3,7,9957,4,10,4,183,8.3,0,0,5,4.666667
9,9,154169,승리,트런들,라인전이기니갱와,1,6,13,5358,6,25,9,47,2.1,0,0,5,2.333333


In [163]:
test_data["Object"] = 3.2*test_data["Baron"] + 1*test_data["Dragon"]
test_data["Ward"] = 0.18*test_data["Buy_Pward"] + test_data["Build_ward"] + 2*test_data["Destroy_ward"]
test_data['KDA2'] = test_data['Kill'] + 0.2*test_data['Assist'] - 1.2*test_data['Death']
sub_test = test_data[["MatchNumber","GameResult","Kill","Death","Assist","Ward","Damage","KDA2","Tower","Object","CS"]]
result = []

for i in sub_test["GameResult"]:
    if i == "승리":
        result.append(1)
    else:
        result.append(0)
        
sub_test["Result"] = result;
sub_test.head(20)

C:\Users\waf\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,MatchNumber,GameResult,Kill,Death,Assist,Ward,Damage,KDA2,Tower,Object,CS,Result
0,154169,패배,0,6,3,9.18,10743,-6.6,1,0.0,112,0
1,154169,패배,2,3,5,39.44,6993,-0.6,1,0.0,101,0
2,154169,패배,3,4,1,20.54,10188,-1.6,1,0.0,187,0
3,154169,패배,5,8,4,18.18,15458,-3.8,1,0.0,150,0
4,154169,패배,2,4,9,21.90,5339,-1.0,1,0.0,45,0
5,154169,승리,5,0,10,28.36,21246,7.0,5,0.0,221,1
6,154169,승리,12,2,4,20.18,12514,10.4,5,0.0,91,1
7,154169,승리,0,1,9,28.54,10661,0.6,5,0.0,156,1
8,154169,승리,7,3,7,18.72,9957,4.8,5,0.0,183,1
9,154169,승리,1,6,13,44.08,5358,-3.6,5,0.0,47,1


In [164]:
def cvt2(parameter):
    index = 0
    result = []
    tmp = []
    for item in sub_test[parameter]:
        index += 1
        tmp.append(item)
        if index % 10 == 0:
            result.append(tmp)
            tmp=[]
    result = np.array(result)
    res = np.ones([result.shape[0],int(result.shape[1]/2)])
    for i in range(len(result)):
        res[i] = result[i][0:5]-result[i][5:10]
    return res

In [165]:
def cvt_sum2(parameter): # 10명의 전체 합에 대한 비율 * 100
    index = 0
    result = []
    tmp = []
    for item in sub_test[parameter]:
        index += 1
        tmp.append(item)
        if index % 10 == 0:
            sum_da = 0
            for j in tmp:
                sum_da += j
            result.append(np.array(tmp)/sum_da*100)
            tmp=[]
    result = np.array(result)
    res = np.ones([result.shape[0],int(result.shape[1]/2)])
    for i in range(len(result)):
        res[i] = result[i][0:5]-result[i][5:10]
    return res

In [166]:
def cvt_result2(parameter): # 결과를 뽑을 때 사용
    index = 0
    result = []
    tmp = []
    for item in sub_test[parameter]:
        index += 1
        if index % 10 == 1:
            tmp.append(item)
        if index % 10 == 6:
            tmp.append(item)
        if index % 10 == 0:
            result.append(tmp)
            tmp=[]
    result = np.array(result)
    return result

In [167]:
kill = cvt2("Kill")
death = cvt2("Death")
assist = cvt2("Assist")
kda2 = cvt2("KDA2")
ward = cvt_sum2("Ward")
damage = cvt_sum2("Damage")
tower = cvt2("Tower")
obj = cvt2("Object")
CS = cvt_sum2("CS")
ylabel = cvt_result2("Result")
actual = np.argmax(ylabel,axis=1)
actual.shape

(1308,)

- 앞의 계수는 머신러닝 결과 정확도를 넣어준 것, 변수로 받아서 자동화 하려고 했으나 문제가 발생했고, 시간상 손으로 붙임

In [168]:
predict = []
for i in range(int(len(sub_test)/10)):
    predict.append(np.argmax(0.9544*sigmoid(np.matmul(kill[i],w1)+b1)+0.9514*sigmoid(np.matmul(death[i],w2)+b2)
                             +0.9331*sigmoid(np.matmul(assist[i],w3)+b3)+0.9608*sigmoid(np.matmul(kda2[i],w4)+b4)
                             +0.7308*sigmoid(np.matmul(ward[i],w5)+b5)+0.8963*sigmoid(np.matmul(damage[i],w6)+b6)
                             +0.9740*sigmoid(np.matmul(tower[i],w7)+b7)+0.8267*sigmoid(np.matmul(obj[i],w8)+b8)
                             +0.8258*sigmoid(np.matmul(CS[i],w9)+b9)))

score = 0
for i in range(len(predict)):
    if predict[i] == actual[i]:
        score += 1
score = score/len(predict)
print("Test_data 정확도 :",score)

Test_data 정확도 : 0.9571865443425076


In [ ]:
# get data file names
filename = "match35.csv"
test_data = pd.read_csv(filename, engine='python')
test_data['Death_fake'] = test_data['Death']
test_data['Death_fake'] = test_data['Death_fake'].replace(0, 1)
test_data['KDA'] = (test_data['Kill'] + test_data['Assist'])/ test_data['Death_fake']
test_data = test_data.drop('Death_fake', 'columns')
test_data["Team"] = [1,1,1,1,1,2,2,2,2,2]
test_data

In [ ]:
predict = []
for i in range(int(len(sub)/10)):
    predict.append(np.argmax(sigmoid(np.matmul(kill[i], w1) + b1)+sigmoid(np.matmul(death[i], w2) + b2)+sigmoid(np.matmul(assist[i], w3) + b3)))

In [ ]:
score = 0
for i in range(len(predict)):
    if predict[i] == actual[i]:
        score += 1
score = score/len(predict)

## 3. Predict
### 1) 개인전적를 임의로 뽑아서 평균 지표를 넣고 머신러닝 결과를 적용 (26 경기)
- 머신러닝 정확도
- Train_data : 0.9719241953273839
- Test_data : 0.9571865443425076
- Predict : 0.6153846153846154
- Predict는 몇 개의 요소를 빼가면서 어떤 것이 중요하나 탐색하였고 그 결과 KDA2와 Ward 점수가 예측에 큰 영향을 주는 것을 확인

In [178]:
# get data file names
path =r'C:\Users\waf\Documents\00_DGIST 수업\03_02\Big data\프로젝트\2017_BigData\Data\match\match\일반'
filenames = glob.glob(path + "/*.csv")
filenames
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, engine='python'))
# Concatenate all data into one DataFrame
test_data = pd.concat(dfs, ignore_index=True)
test_data['Death_fake'] = test_data['Death']
test_data['Death_fake'] = test_data['Death_fake'].replace(0, 1)
test_data['KDA'] = (test_data['Kill'] + test_data['Assist'])/ test_data['Death_fake']
test_data = test_data.drop('Death_fake', 'columns')
test_data["Team"] = [1,1,1,1,1,2,2,2,2,2]*len(dfs)
test_data

,Assist,Baron,Build_ward,Buy_Pward,CS,CSperMin,Damage,Death,Destroy_ward,Dragon,KDA,KDA2,Kill,Nickname,Tower,Team
0,4.518519,0.425926,19.185185,2.166667,189.129630,7.227778,13707.833333,4.833333,7.129630,0.685185,1.816092,0.718519,4.259259,KDM PERT,3.962963,1
1,6.388889,0.305556,24.000000,1.666667,122.555556,4.702778,11653.888889,6.138889,8.388889,0.388889,1.923077,1.244444,5.416667,9346,3.861111,1
2,5.270270,0.270270,15.081081,2.918919,169.675676,6.754054,13428.378378,4.567568,7.864865,0.810811,1.804734,0.127027,2.972973,으아으앙으아앙,4.000000,1
3,7.625000,0.531250,22.937500,4.906250,198.250000,7.759375,20548.218750,5.156250,7.406250,1.031250,2.478788,2.781250,5.156250,deft123456,4.250000,1
4,7.968750,0.687500,10.562500,1.375000,183.250000,5.887500,18383.000000,6.500000,8.687500,1.062500,2.149038,2.184375,6.000000,구름꽃길만걷자,4.812500,1
5,7.395349,0.372093,13.860465,1.720930,191.232558,7.539535,14046.837209,3.558140,3.511628,1.232558,3.405229,4.148837,4.720930,쌀국수뚝붸기,6.348837,2
6,10.548387,0.258065,19.129032,4.548387,115.483871,4.864516,11955.161290,4.806452,7.290323,0.741935,3.355705,5.087097,5.580645,일견무시도성빈,5.612903,2
7,8.900000,0.233333,22.233333,4.366667,217.766667,8.630000,18663.566667,2.566667,7.833333,1.033333,5.064935,5.470000,4.100000,MVP Ian,5.933333,2
8,5.463415,0.390244,13.439024,1.756098,194.780488,7.751220,17078.731707,3.268293,5.268293,1.390244,3.753731,5.614634,6.804878,어껭간만에찾아뵙,7.243902,2
9,8.051282,0.410256,16.205128,4.794872,172.461538,6.017949,16949.076923,5.000000,5.897436,0.923077,2.553846,2.743590,4.717949,마산가자,6.128205,2


In [179]:
test_data["Object"] = 3.2*test_data["Baron"] + 1*test_data["Dragon"]
test_data["Ward"] = 0.18*test_data["Buy_Pward"] + test_data["Build_ward"] + 2*test_data["Destroy_ward"]
test_data['KDA2'] = test_data['Kill'] + 0.2*test_data['Assist'] - 1.2*test_data['Death']
sub_test = test_data[["Team","Kill","Death","Assist","Ward","Damage","KDA2","Tower","Object","CS"]]
sub_test.head(20)

,Team,Kill,Death,Assist,Ward,Damage,KDA2,Tower,Object,CS
0,1,4.259259,4.833333,4.518519,33.834444,13707.833333,-0.637037,3.962963,2.048148,189.129630
1,1,5.416667,6.138889,6.388889,41.077778,11653.888889,-0.672222,3.861111,1.366667,122.555556
2,1,2.972973,4.567568,5.270270,31.336216,13428.378378,-1.454054,4.000000,1.675676,169.675676
3,1,5.156250,5.156250,7.625000,38.633125,20548.218750,0.493750,4.250000,2.731250,198.250000
4,1,6.000000,6.500000,7.968750,28.185000,18383.000000,-0.206250,4.812500,3.262500,183.250000
5,2,4.720930,3.558140,7.395349,21.193488,14046.837209,1.930233,6.348837,2.423256,191.232558
6,2,5.580645,4.806452,10.548387,34.528387,11955.161290,1.922581,5.612903,1.567742,115.483871
7,2,4.100000,2.566667,8.900000,38.686000,18663.566667,2.800000,5.933333,1.780000,217.766667
8,2,6.804878,3.268293,5.463415,24.291707,17078.731707,3.975610,7.243902,2.639024,194.780488
9,2,4.717949,5.000000,8.051282,28.863077,16949.076923,0.328205,6.128205,2.235897,172.461538


- ylabel은 파일로 받는 걸로 했으나 시간상으로 문제로 일단은 손으로 기록

In [180]:
kill = cvt2("Kill")
death = cvt2("Death")
assist = cvt2("Assist")
kda2 = cvt2("KDA2")
ward = cvt_sum2("Ward")
damage = cvt_sum2("Damage")
tower = cvt2("Tower")
obj = cvt2("Object")
CS = cvt_sum2("CS")
actual = [1, 0, 0, 0, 0, 0,
          1, 0, 1, 1, 0,
          0, 0, 1, 1, 0,
          0, 0, 1, 0, 1,
          1, 0, 1, 1, 1]

In [ ]:
    predict.append(np.argmax(0.9544*sigmoid(np.matmul(kill[i],w1)+b1)+0.9514*sigmoid(np.matmul(death[i],w2)+b2)
                             +0.9331*sigmoid(np.matmul(assist[i],w3)+b3)+0.9608*sigmoid(np.matmul(kda2[i],w4)+b4)
                             +0.7308*sigmoid(np.matmul(ward[i],w5)+b5)+0.8963*sigmoid(np.matmul(damage[i],w6)+b6)
                             +0.9740*sigmoid(np.matmul(tower[i],w7)+b7)+0.8267*sigmoid(np.matmul(obj[i],w8)+b8)
                             +0.8258*sigmoid(np.matmul(CS[i],w9)+b9)))

In [199]:
predict = []
for i in range(int(len(sub_test)/10)):
    predict.append(np.argmax(0.9608*sigmoid(np.matmul(kda2[i],w4)+b4)+0.7308*sigmoid(np.matmul(ward[i],w5)+b5)))

score = 0
for i in range(len(predict)):
    if predict[i] == actual[i]:
        score += 1
score = score/len(predict)
print("Predict 정확도 :",score)

Predict 정확도 : 0.6153846153846154


### 2) 응용 분야 : 프로게이머 전적 데이터를 이용하여 팀끼리 경기를 시켜서 나오는 결과 예측 (4개팀, 6경기)